<a href="https://colab.research.google.com/github/AbhinavGunti/NAM_Project/blob/main/Knowledge_Graph_Embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install torchkge

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.6/51.6 kB 2.2 MB/s eta 0:00:00


In [47]:
from torch import cuda
from torch.optim import Adam

from torchkge.models import TransEModel,TransRModel,TransDModel,TransHModel

from torchkge.sampling import BernoulliNegativeSampler
from torchkge.utils import MarginLoss, DataLoader
from torchkge.utils.datasets import load_fb15k

from tqdm.autonotebook import tqdm
import torch

# Dataset

In [24]:
dataset=load_fb15k()

# TransE

In [ ]:
# Load dataset
kg_train, _, _ = load_fb15k()

# Define some hyper-parameters for training
emb_dim = 100
lr = 0.0004
n_epochs = 1000
b_size = 32768
margin = 0.5

# Define the model and criterion
model_TransE = TransEModel(emb_dim, kg_train.n_ent, kg_train.n_rel, dissimilarity_type='L2')
criterion = MarginLoss(margin)
model_TransE

# Move everything to CUDA if available
if cuda.is_available():
    cuda.empty_cache()
    model_TransE.cuda()
    criterion.cuda()
# Define the torch optimizer to be used
optimizer = Adam(model_TransE.parameters(), lr=lr, weight_decay=1e-5)

sampler = BernoulliNegativeSampler(kg_train)
dataloader = DataLoader(kg_train, batch_size=b_size, use_cuda='all')
iterator = tqdm(range(n_epochs), unit='epoch')
for epoch in iterator:
    running_loss = 0.0
    for i, batch in enumerate(dataloader):
        h, t, r = batch[0], batch[1], batch[2]
        n_h, n_t = sampler.corrupt_batch(h, t, r)

        optimizer.zero_grad()

        # forward + backward + optimize
        pos, neg = model_TransE(h, t, r, n_h, n_t)
        loss = criterion(pos, neg)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    iterator.set_description(
        'Epoch {} | mean loss: {:.5f}'.format(epoch + 1,
                                              running_loss / len(dataloader)))

model_TransE.normalize_parameters()

  0%|          | 0/1000 [00:00<?, ?epoch/s]

In [34]:
entities_embeddings_transE=model_TransE.get_embeddings()[0]
relations_embeddings_transE=model_TransE.get_embeddings()[1]
len(entities_embeddings_transE),len(relations_embeddings_transE)
len(entities_embeddings_transE[1]) #embedding dimension : 100

(14951, 1345)

# TransH

In [ ]:
# Load dataset
kg_train, _, _ = load_fb15k()

# Define some hyper-parameters for training
emb_dim = 100
lr = 0.0004
n_epochs = 1000
b_size = 32768
margin = 0.5

# Define the model and criterion
model_TransH = TransEModel(emb_dim, kg_train.n_ent, kg_train.n_rel, dissimilarity_type='L2')
criterion = MarginLoss(margin)
model_TransH

# Move everything to CUDA if available
if cuda.is_available():
    cuda.empty_cache()
    model_TransH.cuda()
    criterion.cuda()
# Define the torch optimizer to be used
optimizer = Adam(model_TransH.parameters(), lr=lr, weight_decay=1e-5)

sampler = BernoulliNegativeSampler(kg_train)
dataloader = DataLoader(kg_train, batch_size=b_size, use_cuda='all')
iterator = tqdm(range(n_epochs), unit='epoch')
for epoch in iterator:
    running_loss = 0.0
    for i, batch in enumerate(dataloader):
        h, t, r = batch[0], batch[1], batch[2]
        n_h, n_t = sampler.corrupt_batch(h, t, r)

        optimizer.zero_grad()

        # forward + backward + optimize
        pos, neg = model_TransH(h, t, r, n_h, n_t)
        loss = criterion(pos, neg)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    iterator.set_description(
        'Epoch {} | mean loss: {:.5f}'.format(epoch + 1,
                                              running_loss / len(dataloader)))

model_TransH.normalize_parameters()

  0%|          | 0/1000 [00:00<?, ?epoch/s]

In [63]:
for i,batch in enumerate(dataloader):
  print(batch[0])
h_idx, t_idx, r_idx = batch[0], batch[1], batch[2]
print(f"Head : {h_idx}\nTail : {t_idx}\nRelation {r_idx}")

tensor([ 3920,   839, 10094,  ...,  2025,   751, 12048], device='cuda:0')
tensor([11326,  4707,  5877,  ...,  1075, 11686,    58], device='cuda:0')
tensor([12914, 13653,  8606,  ...,  5568,   894, 11632], device='cuda:0')
tensor([ 6310, 13426,  2648,  ...,  8743,   718,  3426], device='cuda:0')
tensor([ 4527,  2986,  8805,  ...,  9907, 12944,  1856], device='cuda:0')
tensor([ 9275, 12933, 13989,  ...,  4686,  1070, 11677], device='cuda:0')
tensor([11853,  7585, 12238,  ..., 10726, 12688,   577], device='cuda:0')
tensor([  448,  6560, 13137,  ...,  8178,  8951, 10081], device='cuda:0')
tensor([ 3715,   243, 12234,  ...,  7945,  3575,  9342], device='cuda:0')
tensor([ 8138,  3311, 14526,  ...,  2747,  8735,  9494], device='cuda:0')
tensor([14594,  7255,  3248,  ...,   649,  6169,  1705], device='cuda:0')
tensor([  715,  5174,  5434,  ..., 12031,  4595, 12182], device='cuda:0')
tensor([ 8062,  8163, 12982,  ...,  5968, 10505, 11255], device='cuda:0')
tensor([14192,  5380, 11413,  ...,  48

In [64]:
score_1=model_TransH.scoring_function(h_idx, t_idx, r_idx)
score_1

tensor([-5.9702, -2.7647, -2.8393,  ..., -2.2246, -2.2750, -1.6837],
       device='cuda:0', grad_fn=<NegBackward0>)

In [48]:
torch.tensor(1)

tensor(1)

# TransD

In [38]:
# # Load dataset
# kg_train, _, _ = load_fb15k()

# # Define some hyper-parameters for training
# emb_dim = 100
# lr = 0.0004
# n_epochs = 1000
# b_size = 32768
# margin = 0.5

# # Define the model and criterion
# model_TransD = TransDModel(emb_dim, kg_train.n_ent, kg_train.n_rel, dissimilarity_type='L2')
# criterion = MarginLoss(margin)
# model_TransD

# # Move everything to CUDA if available
# if cuda.is_available():
#     cuda.empty_cache()
#     model_TransD.cuda()
#     criterion.cuda()
# # Define the torch optimizer to be used
# optimizer = Adam(model_TransD.parameters(), lr=lr, weight_decay=1e-5)

# sampler = BernoulliNegativeSampler(kg_train)
# dataloader = DataLoader(kg_train, batch_size=b_size, use_cuda='all')
# iterator = tqdm(range(n_epochs), unit='epoch')
# for epoch in iterator:
#     running_loss = 0.0
#     for i, batch in enumerate(dataloader):
#         h, t, r = batch[0], batch[1], batch[2]
#         n_h, n_t = sampler.corrupt_batch(h, t, r)

#         optimizer.zero_grad()

#         # forward + backward + optimize
#         pos, neg = model_TransD(h, t, r, n_h, n_t)
#         loss = criterion(pos, neg)
#         loss.backward()
#         optimizer.step()

#         running_loss += loss.item()
#     iterator.set_description(
#         'Epoch {} | mean loss: {:.5f}'.format(epoch + 1,
#                                               running_loss / len(dataloader)))

# model_TransD.normalize_parameters()

# TransR